In [ ]:
# Path to Kaggle.json file
json_path = "~/Downloads/kaggle.json"

In [ ]:
!mkdir ~/.kaggle
!mv {json_path} ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jessicali9530/celeba-dataset

In [ ]:
!mkdir data
!unzip celeba-dataset.zip -d data
!unzip data/img_align_celeba.zip

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import matplotlib.pyplot as plt
import cv2 
import os
import time

print(tf.__version__)

!mkdir celeba_gen_images/
!mkdir checkpoints

In [ ]:
# Shape of images
SHAPE = (64, 64)

In [ ]:
img = cv2.imread("data/img_align_celeba/img_align_celeba/000012.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, SHAPE)
plt.imshow(img)

In [ ]:
NUM_IMAGES = 10000

def open_all_images(path):
    image_names = os.listdir(path)
    X = []
    for img_id in range(NUM_IMAGES):
        image_path = os.path.join(path, image_names[img_id])
    
        img = cv2.imread(image_path, -1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, SHAPE)

        assert img.shape == (SHAPE[0], SHAPE[1], 3)
        X.append(img)

        if img_id % 5000 == 0:
            print(f"Image {img_id} out of {len(image_names)}")

    
    return ( (np.array(X) - np.array(X).min()) / (255 - np.array(X).min()) ) *2 - 1

x_train = open_all_images("data/img_align_celeba/img_align_celeba")
print(x_train.shape)
print("MIN:", x_train.min(), "MAX:", x_train.max())

In [ ]:
def discriminator(X, reuse=False):
    with tf.variable_scope("discriminator", reuse=reuse):
        X = tf.layers.conv2d(X, 32, 5, 2, padding="same")
        X = tf.nn.leaky_relu(X, 0.2)

        X = tf.layers.conv2d(X, 64, 5, 2, padding="same")
        X = tf.layers.batch_normalization(X)
        X = tf.nn.leaky_relu(X, 0.2)

        X = tf.layers.conv2d(X, 128, 3, 2, padding="same")
        X = tf.layers.batch_normalization(X)
        X = tf.nn.leaky_relu(X, 0.2)

        X = tf.layers.conv2d(X, 256, 3, 2, padding="same")
        X = tf.layers.batch_normalization(X)
        X = tf.nn.leaky_relu(X, 0.2)

        X = tf.layers.flatten(X)
        X = tf.layers.dropout(X, 0.4)

        out = tf.layers.dense(X, 1)

        return out

In [ ]:
def generator(X, reuse=False):
    with tf.variable_scope("generator", reuse=reuse):
        X = tf.layers.dense(X, units=2*2*1024)
        X = tf.layers.batch_normalization(X)
        X = tf.reshape(X, shape=[-1, 2, 2, 1024])

        X = tf.layers.conv2d_transpose(X, 512, 5, 2, padding="same")
        X = tf.layers.batch_normalization(X)
        X = tf.nn.relu(X)

        X = tf.layers.conv2d_transpose(X, 256, 5, 2, padding="same")
        X = tf.layers.batch_normalization(X)
        X = tf.nn.relu(X)

        X = tf.layers.conv2d_transpose(X, 128, 5, 2, padding="same")
        X = tf.layers.batch_normalization(X)
        X = tf.nn.relu(X)

        X = tf.layers.conv2d_transpose(X, 64, 3, 2, padding="same")
        X = tf.layers.batch_normalization(X)
        X = tf.nn.relu(X)

        X = tf.layers.conv2d_transpose(X, 32, 3, 2, padding="same")
        X = tf.layers.batch_normalization(X)
        X = tf.nn.relu(X)

        X = tf.layers.conv2d(X, 3, 1, padding="same")
        out = tf.nn.tanh(X)

        return out

In [ ]:
# Create Placeholders
def get_inputs():
    real_in = tf.placeholder(tf.float32, shape=[None, SHAPE[0], SHAPE[1], 3])
    noise_in = tf.placeholder(tf.float32, shape=[None, 100])
    return real_in, noise_in

In [ ]:
from tensorflow.nn import sigmoid_cross_entropy_with_logits as loss

# Parameters
smooth = 0.9
lr = 0.0002
b1 = 0.5

tf.reset_default_graph()

real_in, noise_in = get_inputs()

gen_sample = generator(noise_in)
dis_real = discriminator(real_in)
dis_fake = discriminator(gen_sample, reuse=True)

dis_real_loss = tf.reduce_mean(loss(logits=dis_real, labels=smooth * tf.ones_like(dis_real)))
dis_fake_loss = tf.reduce_mean(loss(logits=dis_fake, labels=tf.zeros_like(dis_fake)))
gen_loss = tf.reduce_mean(loss(logits=dis_fake, labels=smooth * tf.ones_like(dis_fake)))

dis_loss = dis_real_loss + dis_fake_loss

total_vars = tf.trainable_variables()

gen_vars = [var for var in total_vars if var.name[0] == "g"]
dis_vars = [var for var in total_vars if var.name[0] == "d"]

gen_train = tf.train.AdamOptimizer(lr, beta1=b1).minimize(gen_loss, var_list=gen_vars)
dis_train = tf.train.AdamOptimizer(lr, beta1=b1).minimize(dis_loss, var_list=dis_vars)

In [ ]:
"""
EPOCHS = 100
BATCH_SIZE = 128

saver = tf.train.Saver()

def get_random_values(shape):
    return np.random.normal(0, 1, size=shape)

with tf.Session() as sess:
  init = tf.global_variables_initializer()
  sess.run(init)
  
  print(f"Training {x_train.shape[0]} examples for {EPOCHS} epochs")
    
  for epoch in range(EPOCHS):
    num_batches = x_train.shape[0] // BATCH_SIZE
    
    for batch in range(num_batches):
      x_batch = x_train[BATCH_SIZE*batch : BATCH_SIZE*(batch+1)]      
      n_values = get_random_values((BATCH_SIZE, 100)) 
      
      sess.run(dis_train, feed_dict={real_in: x_batch, noise_in: n_values})
      sess.run(gen_train, feed_dict={real_in: x_batch, noise_in: n_values})
    
    g_loss, d_loss = sess.run([gen_loss, dis_loss], feed_dict={real_in: x_batch, noise_in: n_values})
    print("Epoch: {}/{}; d_loss={:.4f}, g_loss={:.4f}".format(epoch, EPOCHS, d_loss, g_loss))
    
    saved_path = saver.save(sess, "./checkpoints/model.ckpt")
    saver.export_meta_graph("checkpoints/model.ckpt", gen_vars)
    print("Checkpoint saved on:", saved_path)
    
    if epoch % 5 == 0:
      sample_noise = get_random_values((2, 100))
      sample_generated = sess.run(gen_sample, feed_dict={noise_in: sample_noise})
      
      for sample_id, sample in enumerate(sample_generated):
        cv2.imwrite("celeba_gen_images/image-{}-{}.png".format(epoch, sample_id), sample)
      
      f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True)
      ax1.imshow(sample_generated[0])
      ax2.imshow(sample_generated[1])
      plt.show()
"""

In [63]:
EPOCHS = 100
BATCH_SIZE = 128

saver = tf.train.Saver()

def get_random_values(shape):
    return np.random.normal(0, 1, size=shape)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    print(f"Training {x_train.shape[0]} examples for {EPOCHS} epochs")
    
    for epoch in range(EPOCHS):
        num_batches = x_train.shape[0]//BATCH_SIZE
        
        for batch in range(num_batches):
            x_batch = x_train[BATCH_SIZE*batch : BATCH_SIZE*(batch+1)]
            n_values = get_random_values((BATCH_SIZE, 100))
            
            sess.run(dis_train, feed_dict={real_in: x_batch, noise_in: n_values})
            sess.run(gen_train, feed_dict={real_in: x_batch, noise_in: n_values})
        
        g_loss, d_loss = sess.run([gen_loss, dis_loss], feed_dict={real_in: x_batch, noise_in: n_values})
        print("Epoch: {}/{}; d_loss={:.4f}, g_loss={:.4f}".format(epoch, EPOCHS, d_loss, g_loss))
        
        saved_path = saver.save(sess, "./checkpoints/model.ckpt")
        saver.export_meta_graph("./checkpoints/model.ckpt", gen_vars)
        print("Checkpoint saved on:", saved_path)
        
        if epoch % 5 == 0:
            sample_noise = get_random_values((2,100))
            sample_generated = sess.run(gen_sample, feed_dict={noise_in: sample_noise})
            
            for sample_id, sample in enumerate(sample_generated):
                cv2.imwrite("celeba_gen_images/image-{}-{}.png".format(epoch, sample_id, sample), sample)
                
            f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True)
            ax1.imshow(sample_generated[0])
            ax2.imshow(sample_generated[1])
            plt.show()

Training 10000 examples for 100 epochs
Epoch: 0/100; d_loss=0.7319, g_loss=3.0571
Checkpoint saved on: ./checkpoints/model.ckpt


TypeError: imwrite() missing required argument 'img' (pos 2)

In [ ]:
!ls checkpoints/

In [ ]:
shape = (1, 100)

n = np.zeros(shape)

place = tf.placeholder(tf.float32, shape=shape)

def conv2d_t(x, filters, kernel, stride):
    return tf.layers.conv2d_transpose(x, filters, kernel, stride, padding="same")

res = tf.layers.dense(place, 2*2*1024)
res = tf.reshape(res, (-1, 2, 2, 1024))
res = conv2d_t(res, 512, 5, 2)
res = conv2d_t(res, 256, 5, 2)
res = conv2d_t(res, 128, 5, 2)
res = conv2d_t(res, 128, 3, 2)
res = conv2d_t(res, 128, 3, 2)
res = conv2d_t(res, 64, 3, 2)
res = conv2d_t(res, 32, 3, 2)
res = tf.layers.conv2d(res, 3, 1,1)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
  
    val = sess.run(res, feed_dict={place: n})
    print(val.shape)

In [ ]:
!ls checkpoints